In [1]:
import pandas as pd
import numpy as np
import seaborn as sb
import matplotlib.pyplot as plt

In [2]:
df = pd.read_csv('cars.csv')
df.head()

,VehicleID,Location,Maker,Model,Year,Colour,Amount (Million Naira),Type,Distance
0,VHL12546,Abuja,Honda,Accord Coupe EX V-6,"2,011",Silver,2.2,Nigerian Used,NaN
1,VHL18827,Ibadan,Hyundai,Sonata,"2,012",Silver,3.5,Nigerian Used,"125,000"
2,VHL19499,Lagos,Lexus,RX 350,"2,010",Red,9.2,Foreign Used,"110,852"
3,VHL17991,Abuja,Mercedes-Benz,GLE-Class,"2,017",Blue,22.8,Foreign Used,"30,000"
4,VHL12170,Ibadan,Toyota,Highlander,"2,002",Red,2.6,Nigerian Used,"125,206"


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7205 entries, 0 to 7204
Data columns (total 9 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   VehicleID               7205 non-null   object 
 1   Location                7205 non-null   object 
 2   Maker                   7205 non-null   object 
 3   Model                   7205 non-null   object 
 4   Year                    7184 non-null   object 
 5   Colour                  7205 non-null   object 
 6   Amount (Million Naira)  7188 non-null   float64
 7   Type                    7008 non-null   object 
 8   Distance                4845 non-null   object 
dtypes: float64(1), object(8)
memory usage: 506.7+ KB


In [4]:
df.shape

(7205, 9)

In [5]:
df.duplicated().sum()

0

In [6]:
df.isnull().sum()

VehicleID                    0
Location                     0
Maker                        0
Model                        0
Year                        21
Colour                       0
Amount (Million Naira)      17
Type                       197
Distance                  2360
dtype: int64

In [7]:
df['Year'] = df['Year'].str.replace(',', '')
df['Distance'] = df['Distance'].str.replace(',', '')

In [8]:
df['Distance'] = df['Distance'].fillna(df['Distance'].median())
df['Year'] = df['Year'].fillna(df['Year'].median())
df['Type'] = df['Type'].fillna(method = 'ffill')
df['Amount (Million Naira)'] = df['Amount (Million Naira)'].fillna(df['Amount (Million Naira)'].mean())

In [9]:
df.isnull().sum()

VehicleID                 0
Location                  0
Maker                     0
Model                     0
Year                      0
Colour                    0
Amount (Million Naira)    0
Type                      0
Distance                  0
dtype: int64

In [10]:
df['Year'] = df['Year'].astype(int)
df['Distance'] = df['Distance'].astype(float)

In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7205 entries, 0 to 7204
Data columns (total 9 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   VehicleID               7205 non-null   object 
 1   Location                7205 non-null   object 
 2   Maker                   7205 non-null   object 
 3   Model                   7205 non-null   object 
 4   Year                    7205 non-null   int32  
 5   Colour                  7205 non-null   object 
 6   Amount (Million Naira)  7205 non-null   float64
 7   Type                    7205 non-null   object 
 8   Distance                7205 non-null   float64
dtypes: float64(2), int32(1), object(6)
memory usage: 478.6+ KB


In [12]:
df.drop(['VehicleID'], axis = 1, inplace = True)

In [13]:
num_col = ['Year', 'Amount (Million Naira)', 'Distance']
# cat_col = df.select_dtypes(include = ['object', 'category']).columns
cat_col = [col for col in df.columns if col not in num_col]
cat_col

['Location', 'Maker', 'Model', 'Colour', 'Type']

In [14]:
from sklearn.preprocessing import LabelEncoder, OneHotEncoder

le = LabelEncoder()
for col in cat_col:
    df[col] = le.fit_transform(df[col])

df.head()

,Location,Maker,Model,Year,Colour,Amount (Million Naira),Type,Distance
0,0,17,117,2011,16,2.2,2,80830.0
1,1,19,1049,2012,16,3.5,2,125000.0
2,2,29,908,2010,15,9.2,1,110852.0
3,0,34,508,2017,2,22.8,1,30000.0
4,1,52,569,2002,15,2.6,2,125206.0


## MODELLING

In [45]:
pip install catboost


     ---------------------------------------- 74.0/74.0 MB 3.0 MB/s eta 0:00:00
     ---------------------------------------- 15.3/15.3 MB 3.0 MB/s eta 0:00:00
     --------------------------------------- 47.0/47.0 kB 69.3 kB/s eta 0:00:00


In [48]:
pip install xgboost


     ---------------------------------------- 89.1/89.1 MB 1.9 MB/s eta 0:00:00


In [50]:
pip install svm

  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'

  Created wheel for svm: filename=svm-0.1.0-py3-none-any.whl size=3483 sha256=8ccf85b7e01d5708badf768e8bde550a136b3ce86dada49965653eb608f5d13f
  Stored in directory: c:\users\sofiyah\appdata\local\pip\cache\wheels\38\1e\67\e4fe09dd9a91d94fb9f7cadee4199cfd8f5616a66359c7e733
Successfully built svm


In [17]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from catboost import CatBoostRegressor
from sklearn.tree import DecisionTreeRegressor
from xgboost import XGBRegressor
from sklearn.svm import SVR
from sklearn.neighbors import KNeighborsRegressor
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge

from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error

In [18]:
# split data into predictors and y
x = df.drop('Amount (Million Naira)', axis = 1)
y = df['Amount (Million Naira)']

In [19]:
# split data into training and test set
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.2, random_state = 42)

x_train.shape, y_train.shape, x_test.shape, y_test.shape

((5764, 7), (5764,), (1441, 7), (1441,))

In [24]:
# create a dictionary for rthe modeels

models = {'Random Forest': RandomForestRegressor(),
          'Gradient Boosting': GradientBoostingRegressor(),
          'CatBoost': CatBoostRegressor(),
          'Linear Regression': LinearRegression(),
          'Decision Tree': DecisionTreeRegressor(),
          'XGB': XGBRegressor(),
          'SVR': SVR(),
          'KNeighbors': KNeighborsRegressor(),
          'Ridge': Ridge()
        }


In [27]:
# create a function to fit and score models

def fit_score(models, X_train, X_test, Y_train, Y_test):
    '''
    function to fit and score machine learning models
    parameters
    ---------------------------------------------
    models: dictionary of all scikit learn machine leraning models to fit and evaluate
    X_train: training set of the predictors to fit into model
    X_test: test set of the predictors to fit into model
    Y_train: training set of the depedent varaiable
    Y_test: test set of dependent variable
    '''
#     define a random seed to make same set of prediction appear each time program is run(for reproducability)
    np.random.seed(0)
#     define a dictionary for the model scores
    model_scores = {}

#     iterate through the models dictionary items
    for name, model in models.items():
#         fit model the training set into each model in the dictionary
        model.fit(X_train, Y_train)
#     get the model score and attach it to each of the model name from model dictionary 
        model_scores[name] = model.score(X_test, Y_test)
    return model_scores
        

In [28]:
# Display the evaluation score of the models
scores = fit_score(models = models,
                  X_train = x_train,
                  Y_train = y_train,
                  X_test = x_test,
                  Y_test = y_test)
scores

Learning rate set to 0.053998
0:	learn: 24.2983169	total: 6.8ms	remaining: 6.79s
1:	learn: 23.6956847	total: 10.6ms	remaining: 5.27s
2:	learn: 23.1109554	total: 13.6ms	remaining: 4.52s
3:	learn: 22.5810758	total: 17.5ms	remaining: 4.37s
4:	learn: 22.0576525	total: 20.3ms	remaining: 4.05s
5:	learn: 21.5584076	total: 22.8ms	remaining: 3.78s
6:	learn: 21.1222890	total: 25.7ms	remaining: 3.65s
7:	learn: 20.6828383	total: 28.4ms	remaining: 3.52s
8:	learn: 20.2614548	total: 31.1ms	remaining: 3.42s
9:	learn: 19.8881385	total: 34.6ms	remaining: 3.42s
10:	learn: 19.5220199	total: 36.9ms	remaining: 3.31s
11:	learn: 19.1633534	total: 39.2ms	remaining: 3.23s
12:	learn: 18.8573081	total: 41.4ms	remaining: 3.14s
13:	learn: 18.5464729	total: 43.6ms	remaining: 3.07s
14:	learn: 18.2728666	total: 45.8ms	remaining: 3.01s
15:	learn: 17.9997280	total: 49.6ms	remaining: 3.05s
16:	learn: 17.7401085	total: 52ms	remaining: 3.01s
17:	learn: 17.5179365	total: 54.2ms	remaining: 2.96s
18:	learn: 17.2970259	total: 

{'Random Forest': 0.8208977642759141,
 'Gradient Boosting': 0.8552852815715293,
 'CatBoost': 0.8852632625230871,
 'Linear Regression': 0.23033920476983893,
 'Decision Tree': 0.596319652350358,
 'XGB': 0.8882181957712008,
 'SVR': 0.051935790985988106,
 'KNeighbors': 0.5632856208586888,
 'Ridge': 0.23033413805280434}

In [31]:
# display scores as a dataframe for proper comparison
compare = pd.DataFrame(scores, index=['R2_score'])
compare

,Random Forest,Gradient Boosting,CatBoost,Linear Regression,Decision Tree,XGB,SVR,KNeighbors,Ridge
R2_score,0.820898,0.855285,0.885263,0.230339,0.59632,0.888218,0.051936,0.563286,0.230334


In [ ]:
# plot scores on a chart for proper comapriosn